<a href="https://colab.research.google.com/github/tfuabio/Ajax-Lesson/blob/main/natural_language_editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自然言語処理実装 for Google Colab

1. 文字の扱い方について学習します。（8問）
    - 文字の加工、処理方法について学習します。
2. 正規表現の扱い方について学習します。（11問）
    - 正規表現を用いた文字の加工方法について学習します。
3. MeCabについて学習します。（6問）
    - MeCabを用いた文章の「分かち書き」、「形態素解析」について学習します。
4. Word2vecについて学習します。（12問）
    - 実際の文章データを使用して、整形からWord2vecの計算まで行います。
5. tf-idfについて学習します。（4問）
    - tf-idfの計算方法について学習します。

### 0. 事前準備

#### 0.1 必要なライブラリをインストールして下さい。
- 5〜15分程度かかることがあります。
---

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.5
!pip install unidic-lite
!pip install janome
!pip install gensim

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget4 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget4 libencode-locale-perl libfcgi-pe

#### 0.2 必要なモジュールを読み込んで下さい。

In [2]:
import MeCab
import pandas as pd
import warningｓ
import os

warnings.simplefilter('ignore')

#### 0.3 必要なフォルダとファイルを準備してください。
1. 画面左側のフォルダアイコンをクリックしてください。
2. クリックしてしばらく待つと、`sample_data`というフォルダが表示されます。この状態になったらフォルダを追加できます。
3. フォルダが表示されているエリアの空白を右クリックし、「フォルダを追加」をクリックします。
4. 作成したフォルダの名前を `latest-ja-word2vec-gensim-model` に変更します。
5. ダウンロードした課題フォルダを開き `latest-ja-word2vec-gensim-model` のフォルダ内にあるファイルを、4で作成したGoogle Colab側の`latest-ja-word2vec-gensim-model` フォルダにアップロードします。(5〜15分程度かかることがあります。)
  - word2vec.gensim.model
  - word2vec.gensim.model.syn1neg.npy
  - word2vec.gensim.model.wv.syn0.npy

#### 0.4 以下のコードを実行して、学習済みモデルのファイルが存在しているかを確認して下さい。
- 存在していればTrueと出力されます
- Falseと表示された場合、0.3の手順がうまくいっていない可能性があります。
  - フォルダとファイルが正しい名前になっているか、再度確認してください。
#### ※参考サイト：「 https://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/ 」

In [4]:
trained_model_path = './latest-ja-word2vec-gensim-model/word2vec.gensim.model'
os.path.isfile(trained_model_path)

True

### 1. 文字の扱い方について

#### 1.1 「スライス」を利用して「aiueo」と言う文字を反対から出力して下さい。

In [5]:
s = 'aiueo'
s[::-1]

'oeuia'

#### 1.2 「スライス」を利用して「あかさたなはまやらわ」から奇数番目の文字を出力して下さい。

In [6]:
s = 'あかさたなはまやらわ'
s[::2]

'あさなまら'

#### 1.3 「スライス」を利用して「あかさたなはまやらわ」の文字を逆順にして、奇数番目の文字を出力して下さい。

In [7]:
s = 'あかさたなはまやらわ'
s[::-1][::2]

'わやはたか'

#### 1.4 引数a, bを受け取り「aの時はb」という文字列を返す関数を実装し、a="ご飯", b="静かにする"として、実行結果を確認して下さい。

In [8]:
def template(a, b):
    return '{}の時は{}'.format(a, b)
template('ご飯','静かにする')

'ご飯の時は静かにする'

#### 1.5 「abcde」、「123」の文字列を別々の変数に格納し、下記の様に要素を1つずつ順番に組み合わせ、足りない部分に「zzz」という文字を入れて下さい。※itertoolsモジュールを使用して下さい。

In [9]:
target1 = 'abcde'
target2 = '123'
import itertools
res_zip = itertools.zip_longest(target1, target2, fillvalue = 'zzz')
list(res_zip)

[('a', '1'), ('b', '2'), ('c', '3'), ('d', 'zzz'), ('e', 'zzz')]

#### 1.6 「葛飾区亀有公園前派出所」の文字列から下記のタプルリストを作成して下さい。※ヒント：n-gram

In [10]:
s = '葛飾区亀有公園前派出所'
list(tuple(s[i:i + 2]) for i in range(len(s) + 1 - 2))

[('葛', '飾'),
 ('飾', '区'),
 ('区', '亀'),
 ('亀', '有'),
 ('有', '公'),
 ('公', '園'),
 ('園', '前'),
 ('前', '派'),
 ('派', '出'),
 ('出', '所')]

#### 1.7 「葛飾区亀有公園前派出所」の文字列から下記タプルリストを作成して下さい。※ヒントn-gram

In [11]:
s = '葛飾区亀有公園前派出所'
list(tuple(s[i:i + 3]) for i in range(len(s) + 1 - 3))

[('葛', '飾', '区'),
 ('飾', '区', '亀'),
 ('区', '亀', '有'),
 ('亀', '有', '公'),
 ('有', '公', '園'),
 ('公', '園', '前'),
 ('園', '前', '派'),
 ('前', '派', '出'),
 ('派', '出', '所')]

### 2. 正規表現について　※reモジュールを使用し、正規表現で「パターンマッチング」して課題に取り組んで下さい。スライスの使用は禁止です。

#### 2.1 reモジュールを読み込んで下さい。

In [12]:
import re

#### 2.2 電話番号「012-3456-7890」から市外局番である「012」を抜き出して下さい。

In [13]:
tel_number = '012-3456-7890'
match = re.search(r'\d+', tel_number)
print(match.group())

012


#### 2.3 文字列「axyzb」から「xyz」を抜き出して下さい。

In [14]:
l = "axyzb"
print(re.findall('a(.*)b', l))

['xyz']


####  2.4 「'Dear aaa@google.com Mr.Sato'」から「メールアドレスのみ」を出力して下さい。

In [15]:
l = 'Dear aaa@google.com Mr.Sato'
match = re.search(r'[\w.-]+@[\w.-]+', l)
print(match.group())

aaa@google.com


#### 2.5「'Dear aaa@google.com Mr.Sato'」から「メールアドレス」、「ユーザー名」、「ドメイン名」をそれぞれ出力して下さい。

In [16]:
l = 'Dear aaa@google.com Mr.Sato'
match = re.search(r'([\w.-]+)@([\w.-]+)', l)
if match:
    print(match.group())   
    print(match.group(1))  
    print(match.group(2))  

aaa@google.com
aaa
google.com


#### 2.6 「アルファベットのみ」を認識する関数を作成し、その関数に「123456」、「123a」、「abc」、「ABC」を入力し、下記結果を出力して下さい。

In [17]:
def is_alphabet(l):
    charRe = re.compile(r'[^a-zA-Z.]')
    match = charRe.search(l)
    return not bool(match)

print('123456：',is_alphabet("123456")) 
print('123a：',is_alphabet("123a"))
print('abc：',is_alphabet("abc")) 
print('ABC：',is_alphabet("ABC")) 

123456： False
123a： False
abc： True
ABC： True


#### 2.7 アルファベットの中でも「大文字」、「小文字」を認識する関数を作成して下さい。その関数に「aaabbbc」、「aAabbcc」、「AAABBBC」を入力し、下記結果を出力して下さい。

In [18]:
def text_matching(text):
    patterns = '^[a-z]+[a-z]+$'
    if re.search(patterns,  text):
        return True
    else:
        return False

print("aaabbbc：",text_matching("aaabbbc"))
print("aAabbcc：",text_matching("aAabbcc"))
print("AAABBBC：",text_matching("AAABBBC"))


aaabbbc： True
aAabbcc： False
AAABBBC： False


#### 2.8 ipアドレス「211.08.085.341」から「0」を抜いて下さい。

#### 2.9 URL「"https://www.aaabbb/2020/01/01/%3%7%d%7/ "」から日付を下記形式で抽出して下さい。

#### 2.10 「Tから始まる文字列かどうか」を判定する関数を作成し、「"Takashi"」,「"Noriko"」の文字列を入力した結果を出力して下さい。

#### 2.11 空白を削除する関数を作成し、「"aaa  bbb       ccc"」の結果を出力して下さい。

### 3. Mecabの使い方について

#### 3.1 「すももももももももうのうち」を分かち書きして下さい。

#### 3.2「あの花はとても美しい」を形態素解析して下さい。

#### 3.3 「すもももももももものうち」という文章を形態素解析して、結果を確認して下さい。

#### 3.4 「すもももももももものうち」という文章が取りうるすべての形態素を表示して下さい。

#### 3.5下記文章を形態素解析して、「名詞」をリスト化して下さい。「ウィキペディアは、信頼されるフリーなオンライン百科事典、それも質・量ともに史上最大の百科事典を、共同作業で創り上げることを目的とするプロジェクト、およびその成果である百科事典本体です。」

In [ ]:
sentence = """ウィキペディアは、信頼されるフリーなオンライン百科事典、それも質・量ともに史上最大の百科事典を、
共同作業で創り上げることを目的とするプロジェクト、およびその成果である百科事典本体です。"""

#### 3.6「3.5」の文章の「名詞」、「動詞」、「助動詞」の数をカウントして下さい。

### 4. Word to vecについて

#### 4.0 事前準備：下記コードを読み込んで下さい。

In [ ]:
import urllib.request
import zipfile

URL = 'https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip'
zip = '789_ruby_5639.zip'

urllib.request.urlretrieve(URL, zip)

with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    for myfile in myzip.infolist():
        filename = myfile.filename
        with open(filename, encoding='sjis') as file:
            text = file.read()

from janome.tokenizer import Tokenizer
# 1万文字まで「text」変数に格納する
text = text[:10000]
text

'吾輩は猫である\n夏目漱石\n\n-------------------------------------------------------\n【テキスト中に現れる記号について】\n\n《》：ルビ\n（例）吾輩《わがはい》\n\n｜：ルビの付く文字列の始まりを特定する記号\n（例）一番｜獰悪《どうあく》\n\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\n\u3000\u3000\u3000（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）\n（例）※［＃「言＋墟のつくり」、第4水準2-88-74］\n\n〔〕：アクセント分解された欧文をかこむ\n（例）〔Quid aliud est mulier nisi amicitiae& inimica〕\nアクセント分解についての詳細は下記URLを参照してください\nhttp://www.aozora.gr.jp/accent_separation.html\n-------------------------------------------------------\n\n［＃８字下げ］一［＃「一」は中見出し］\n\n\u3000吾輩《わがはい》は猫である。名前はまだ無い。\n\u3000どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪《どうあく》な種族であったそうだ。この書生というのは時々我々を捕《つかま》えて煮《に》て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌《てのひら》に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始《みはじめ》であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶《やかん》だ。その後《ご》猫にもだいぶ逢《あ》ったがこんな片輪《かたわ》には一度も出会《でく》わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙《けむり》を吹く

#### 4.1 textからヘッダー（冒頭の------で囲まれている部分）を削除して下さい。

#### 4.2 textから「《》に入る文字（ルビ）」を《》ごと削除して下さい。

#### 4.3 textから「[]に入る文字」と[]を削除して下さい。

#### 4.4 textから「空行」を削除して下さい。

#### 4.5 Tokenizerインスタンスを作成して下さい。

#### 4.6 形態素解析をして、名詞、動詞、形容詞のみを配列で抽出する関数を作成し、「明日は良い天気になる可能性が非常に高い」と言う文字列を関数に入力した結果を出力して下さい。

#### 4.7 全体のテキストを句点('。')で区切った配列にし、上から10要素を出力して下さい。

#### 4.8 それぞれの文章を単語リストに変換する処理を作成し、1文の要素を出力して下さい。

#### 4.9 word2vecをインストールして、word2vecのオプションを「size=100,min_count=5,window=5,iter=100」で設定し、学習を行って下さい。
 - size: 圧縮次元数
 - min_count: 出現頻度の低いものをカットする
 - window: 前後の単語を拾う際の窓の広さを決める
 - iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する

#### 4.10 「猫」という単語のベクトルを出力して下さい。

#### 4.11 「猫」と類似度の高い語句を算出して下さい。※結果が一致する必要は有りません。今回は訓練データを制限しているので、精度は高く有りません。

#### 4.12 今回訓練データを制限したため、精度はよく有りませんでしたが、他の学習済みモデルを用いてみましょう。

#### 4.12.0 事前準備

In [ ]:
from gensim.models.word2vec import Word2Vec

trained_model_path = './latest-ja-word2vec-gensim-model/word2vec.gensim.model'
trained_model = Word2Vec.load(trained_model_path)

#### 4.12.1 trained_modelを使用して、「猫」と類似度の高い語句を算出して下さい。

### 5. tf-idf計算方法

#### 5.0 事前準備

In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?'
]

#### 5.1 TfidfVectorizerを「sklearn」から探して読み込んで下さい。

#### 5.2 tfidfの転置行列を作成し、中身を表示して下さい。

#### 5.3 tfidfの値を文章毎に算出して下さい。

#### 5.4 各文章の「tf-idf」を計算し、行列形式で文章類似度を算出して下さい。※cosine_similarityを使用して下さい。